In [237]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time

In [239]:
 executable_path = {'executable_path': 'drivers/chromedriver.exe'}
 browser = Browser('chrome', **executable_path, headless=False)

In [240]:
base_url = 'https://mars.nasa.gov/news/'
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
img_url_base = 'https://www.jpl.nasa.gov'
twitter_url = 'https://twitter.com/marswxreport?lang=en'
facts_url = 'http://space-facts.com/mars/'
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [252]:
#Visiting main page
browser.visit(base_url)
html = browser.html

#Getting data
soup = BeautifulSoup(html, "html.parser")

In [253]:
#Getting the 1st title
title = soup.find('div', class_='content_title')
news_title = title.text
print(news_title)

After a Reset, Curiosity Is Operating Normally


In [254]:
#Getting paragraph text from 1st title
para = soup.find('div', class_='article_teaser_body')
news_p = para.text
print(news_p)

NASA's Mars rover Curiosity is in good health but takes a short break while engineers diagnose why it reset its computer. 


In [255]:
#Visiting URL to get the image
browser.visit(img_url)
#Clicking on 'Full Image'
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(1)

In [256]:
#Clicking on 'More Info'
browser.click_link_by_partial_text('more info')
time.sleep(1)

In [257]:
#Getting data
soup = BeautifulSoup(browser.html, "html.parser")
#Image daa
image = soup.find('img', class_='main_image')
print(image)

<img alt="What's that bright point of light in the outer A ring? It's a star, bright enough to be visible through the ring as seen by NASA's Cassini spacecraft." class="main_image" src="/spaceimages/images/largesize/PIA18297_hires.jpg" title="What's that bright point of light in the outer A ring? It's a star, bright enough to be visible through the ring as seen by NASA's Cassini spacecraft."/>


In [258]:
#Image URL
featured_image_url = img_url_base + image["src"]
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18297_hires.jpg


In [259]:
#Getting data from Twitter page
browser.visit(twitter_url)
soup = BeautifulSoup(browser.html, "html.parser")

In [260]:
#Getting the tweet
tweet = soup.find('p', class_='tweet-text')
mars_weather = tweet.text
print(mars_weather)

InSight sol 88 (2019-02-25) low -95.2ºC (-139.4ºF) high -17.8ºC (0.0ºF)
winds from the SW at 5.2 m/s (11.6 mph) gusting to 19.8 m/s (44.3 mph)
pressure at 7.20 hPapic.twitter.com/1msjBvhiu7


In [261]:
#Curiosity was not working, so taken alternative route to get the data apart from daily tweet
#Searching for the URL to go to insight page
w_list = soup.find_all('a', class_="u-textUserColor")

for w in w_list:
    if w.text.strip() == "mars.nasa.gov/news/8415/insi…":
        weather_url = w["title"]
        
print(weather_url)

https://mars.nasa.gov/news/8415/insight-is-the-newest-mars-weather-service/


In [262]:
#Getting into Weather insight page
browser.visit(weather_url)
soup = BeautifulSoup(browser.html, "html.parser")

find_url = soup.find('a', text='a daily weather report')

weather_url_final = find_url["href"]

print(weather_url_final)

https://mars.nasa.gov/insight/weather/


In [263]:
#Getting into Final Weather page
browser.visit(weather_url_final)
soup = BeautifulSoup(browser.html, "html.parser")


In [270]:
#Getting the latest Sol
data = soup.find_all('span', class_='dateSol')
mars_weather = data[-1].text
print(mars_weather)

Sol 88


In [271]:
#Getting the date and adding to the text
sol_table = soup.find_all('th', class_="sol")
mars_weather = "Insight " + mars_weather + " (" + sol_table[2].text + ")"
print(mars_weather)

Insight Sol 88 (Feb. 24, 2019)


In [272]:
#Getting the temparatures
f_table = soup.find_all('span', class_="fahrenheit")
c_table = soup.find_all('span', class_="celsius")
mars_weather = mars_weather + ";  Temp: High " + c_table[2].text + "/" + f_table[2].text + ", Low " + c_table[4].text + "/" + f_table[4].text
print(mars_weather)

Insight Sol 88 (Feb. 24, 2019);  Temp: High -18° C/0° F, Low -95° C/-139° F


In [273]:
#Getting the wind speed
mph_table = soup.find_all('span', class_="mph")
mps_table = soup.find_all('span', class_="mps")
wind_direction = soup.find('td', class_="windspeed point")
mars_weather = mars_weather + "; Wind Speed: Max " + mps_table[0].text + "/" + mph_table[0].text + ", Min " + mps_table[2].text + "/" + mph_table[2].text + ", Direction " + wind_direction.text
print(mars_weather)

Insight Sol 88 (Feb. 24, 2019);  Temp: High -18° C/0° F, Low -95° C/-139° F; Wind Speed: Max 19.8/44.3, Min 0.4/0.8, Direction WNW


In [274]:
#Getting the avg pressure
avg_pressure = soup.find_all('td', class_="pressure avg")
mars_weather = mars_weather + "; Average Pressure " + avg_pressure[1].text + " Pa"

#Final Mars Weather statement from alternate source -
print(mars_weather)

Insight Sol 88 (Feb. 24, 2019);  Temp: High -18° C/0° F, Low -95° C/-139° F; Wind Speed: Max 19.8/44.3, Min 0.4/0.8, Direction WNW; Average Pressure 724.3 Pa


In [275]:
#Getting Mars Facts
facts = pd.read_html(fact_url)
df_facts = facts[0]
df_facts.columns = ["Items", "Values"]
df_facts

,Items,Values
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [276]:
#Creating HTML Table
facts_html = df_facts.to_html()
facts_html = facts_html.replace('\n', '')
facts_html

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Items</th>      <th>Values</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers<

In [277]:
#Getting Hemisphere details
#Create empty bucket
hemisphere_image_urls = []
#Visiting the site and getting data using BeautifulSoup
browser.visit(hemi_url)
soup = BeautifulSoup(browser.html, 'lxml')
#Fetching all items
items = soup.find_all(class_='item')

for item in items:
    browser.visit(hemi_url)
    browser.click_link_by_partial_text(item.h3.text)
    soup = BeautifulSoup(browser.html, 'lxml')
    
    title = item.h3.text.replace(' Enhanced', '')
    img_url = soup.find(target='_blank')['href']
    
    hemisphere_image_urls.append({'title':title, 'img_url':img_url})
    
hemisphere_image_urls


[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [278]:
browser.quit()